# Imports

In [15]:
import torch
import random
import torch.nn as nn
import cv2
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import ResNet50_Weights
from torch.utils.data import random_split
import copy
import numpy as np
import os
import shutil
from tqdm import tqdm
import zipfile
import urllib.request

In [16]:
np.random.seed(123)  # for replication
os.makedirs('./models', exist_ok=True)

# Download the dataset

In [17]:
def download_file(url, file_name):
    if not os.path.exists('dataset/' + file_name):
        with urllib.request.urlopen(url) as response, open('dataset/' + file_name, 'wb') as out_file:
            content_length = int(response.headers['Content-Length'])
            with tqdm(total=content_length, unit='B', unit_scale=True, desc=url.split('/')[-1]) as pbar:
                while True:
                    chunk = response.read(1024)
                    if not chunk:
                        break
                    out_file.write(chunk)
                    pbar.update(len(chunk))
    else:
        print(f"{file_name} already exists.")


os.makedirs("dataset/", exist_ok=True)
# Training
download_file('https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip',
              'GTSRB_Final_Training_Images.zip')
# Testing
download_file('https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip',
              'GTSRB_Final_Test_Images.zip')
# Ground truth
download_file('https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip',
              'GTSRB_Final_Test_GT.zip')

GTSRB_Final_Training_Images.zip already exists.
GTSRB_Final_Test_Images.zip already exists.
GTSRB_Final_Test_GT.zip already exists.


## Extract zip files

In [18]:
def extract_file(file_name):
    with zipfile.ZipFile(f"dataset/{file_name}", 'r') as zip_ref:
        file_list = zip_ref.namelist()
        with tqdm(total=len(file_list), desc="Extracting") as pbar:
            for file in file_list:
                zip_ref.extract(file, 'dataset/')
                pbar.update(1)


extract_file('GTSRB_Final_Training_Images.zip')
extract_file('GTSRB_Final_Test_Images.zip')
extract_file('GTSRB_Final_Test_GT.zip')

Extracting: 100%|██████████| 1/1 [00:00<00:00, 250.00it/s]


## Loading CSV file

In [19]:
#IMAGES: './dataset/GTSRB/test_images'
#CSV ANNOTATIONS: './dataset/GTSRB/test_images/GT-final_test.csv'
def csv_loader(csv_path):
    data = np.loadtxt(csv_path,
                      delimiter=";", dtype=str, skiprows=1)
    return data


#You should download the testset ('GTSRB_Final_Test_Images.zip') from the website which contains only the images
#Then you have to download the ground truth csv ('GTSRB_Final_Test_GT.zip') from the website and paste it into the testset images folder
annotations = csv_loader('./dataset/GT-final_test.csv')
#sort the annotations
annotations = annotations[:, [0, 7]]
num_samples = len(annotations)
#Column 0: filename - Column 1: classid
annotations = annotations[annotations[:, 1].astype(int).argsort()]

## Making training data accordingly

In [20]:
def move_directories(source, destination):
    if not os.path.exists(destination):
        os.makedirs(destination)
    # Get a list of all directories in the source directory
    directories = [d for d in os.listdir(source) if os.path.isdir(os.path.join(source, d))]

    # Move each directory to the destination
    for directory in tqdm(directories):
        source_path = os.path.join(source, directory)
        destination_path = os.path.join(destination, directory)
        shutil.move(source_path, destination_path)


# Move directories with contents
move_directories("./dataset/GTSRB/Final_Training/Images", "./dataset/GTSRB/train")
shutil.rmtree("./dataset/GTSRB/Final_Training")

  0%|          | 0/43 [00:00<?, ?it/s]


Error: Destination path './dataset/GTSRB/train\00000\00000' already exists

## Making test data accordingly

In [ ]:
for class_id in tqdm(np.unique(annotations[:, 1]), desc='Class_ID'):
    newpath = './dataset/GTSRB/test/' + class_id.zfill(5)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for image_filename in annotations[annotations[:, 1] == class_id]:
        shutil.move('./dataset/GTSRB/Final_Test/Images/' + image_filename[0], newpath + '/' + image_filename[0])

shutil.rmtree("./dataset/GTSRB/Final_Test")

# Helper functions

In [ ]:
def plot_classes_preds(images, labels, preds, probs):
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(15, 5))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx + 1, xticks=[], yticks=[])
        norm_img = cv2.normalize(images[idx].cpu().numpy(), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        rgb_img = np.transpose(norm_img, (1, 2, 0)).astype(np.uint8)
        plt.imshow(rgb_img)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            preds[idx],
            probs[idx] * 100.0,
            labels[idx]),
            color=("green" if preds[idx] == labels[idx].item() else "red"))
    return fig

### Loading the train dataset

In [ ]:
# Directory containing the train dataset
train_dir = './dataset/GTSRB/train'

# Load the dataset using the torchvision.datasets.ImageFolder
train_dataset = datasets.ImageFolder(train_dir, ResNet50_Weights.IMAGENET1K_V2.transforms())
train_size = len(train_dataset)
class_names = train_dataset.classes

print('Train size:', train_size)
print('Class names:', class_names)

# Use numpy to count occurrences of each class index in targets
counts = np.bincount(train_dataset.targets)

# Create histogram
plt.figure(figsize=(10, 5))
plt.bar(class_names, counts, color='blue')
plt.xlabel('Classes')
plt.ylabel('Number of images')
plt.title('Number of Images per Class')
plt.xticks(rotation=90)  # Rotate class names for better visibility if needed
plt.yticks(np.arange(0, max(counts) + 10, 250))  # Adjust the range and step size as needed
plt.show()

### Loading the test dataset

In [ ]:
# Directory containing the test dataset
test_dir = './dataset/GTSRB/test'

# Load the dataset using the torchvision.datasets.ImageFolder
test_dataset = datasets.ImageFolder(test_dir, transform=ResNet50_Weights.IMAGENET1K_V2.transforms())
test_size = len(test_dataset)
class_names = test_dataset.classes

print('Test size:', test_size)
print('Class names:', class_names)

# Use numpy to count occurrences of each class index in targets
counts = np.bincount(test_dataset.targets)

# Create histogram
plt.figure(figsize=(10, 5))
plt.bar(class_names, counts, color='blue')
plt.xlabel('Classes')
plt.ylabel('Number of images')
plt.title('Number of Images per Class')
plt.xticks(rotation=90)  # Rotate class names for better visibility if needed
plt.yticks(np.arange(0, max(counts) + 10, 50))  # Adjust the range and step size as needed
plt.show()


# Defining the training phase

In [ ]:
def test_model(trained_model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        trained_model.eval()
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = trained_model(images)
            softmax_outputs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(softmax_outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = float(correct / total)
    print('{} Model accuracy: {:.4f}'.format('Test phase - ', test_accuracy))
    writer.add_scalar('Training/Test Accuracy', test_accuracy)
    return test_accuracy


def train_model(device, model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=25,
                model_name='trained_model'):
    since = time.time()
    time_train = 0
    time_val = 0

    # Save the initial model
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # Choose the appropriate data loader
            if phase == 'train':
                model.train()  # Set model to training mode
                data_total_steps = len(train_loader)
                data_loader = train_loader
            else:
                model.eval()  # Set model to evaluate mode
                data_total_steps = len(val_loader)
                data_loader = val_loader

            for i, (images, labels) in enumerate(data_loader):
                # time_t = epoch * len(data_loader) * i + i

                images = images.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    softmax_outputs = F.softmax(outputs, dim=1)
                    probs, preds = torch.max(softmax_outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * images.size(0)
                running_corrects += torch.sum(preds == labels.data)

                #prints the stats every 20 steps (20 batches performed)
                if (i + 1) % int(data_total_steps / 8) == 0:
                    print(
                        f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{data_total_steps}], Loss: {loss.item():.4f}')

                    # Log image predictions
                    selected_indices = random.sample(range(len(images)), 4)  # Select 4 random indices
                    selected_images = images[selected_indices]
                    selected_labels = labels[selected_indices]
                    selected_preds = preds[selected_indices]
                    selected_probs = probs[selected_indices]
                    if phase == 'train':
                        writer.add_figure('Training/Training Predictions',
                                          plot_classes_preds(selected_images, selected_labels, selected_preds,
                                                             selected_probs),
                                          global_step=time_train)
                    else:
                        writer.add_figure('Training/Validation Predictions',
                                          plot_classes_preds(selected_images, selected_labels, selected_preds,
                                                             selected_probs),
                                          global_step=time_val)

                # Log scalars
                if phase == 'train':
                    writer.add_scalar('Training/Training Loss',
                                      loss.item(),
                                      time_train)
                    writer.add_scalar('Policy/Learning Rate',
                                      np.array(scheduler.get_last_lr()),
                                      time_train)
                    time_train += 1
                else:
                    writer.add_scalar('Training/Validation Loss',
                                      loss.item(),
                                      time_val)
                    time_val += 1

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            if phase == 'train':
                print('{} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(
                    'Train phase - ', epoch + 1, epoch_loss, epoch_acc))
                writer.add_scalar('Training/Training Accuracy',
                                  epoch_acc,
                                  epoch)
                if (epoch + 1) % max(int(num_epochs / 5), 1) == 0:  # checkpoint the model
                    print("----> model checkpoint...")
                    torch.save(model, f'./models/trained_model_{model_name}_epoch_{epoch + 1}.pth')
            else:
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                print('{} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(
                    'Validation phase - ', epoch + 1, epoch_loss, epoch_acc))
                writer.add_scalar('Training/Validation Accuracy',
                                  epoch_acc,
                                  epoch)
                # Uncomment to use scheduler
                #scheduler.step()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc:{:.4f}'.format(best_acc))
    # Return the model with the best accuracy in the validation
    model.load_state_dict(best_model_wts)
    
    return model


def create_dynamic_network(num_features, num_classes, neuron_list=None, dropout_values=None):
    if neuron_list is None:
        neuron_list = []
    layers = []
    num_layers = len(neuron_list)
    # Input layer to first hidden layer
    if num_layers > 0:
        layers.append(nn.Linear(num_features, neuron_list[0]))
        layers.append(nn.ReLU())
        if dropout_values[0] != 0:
            layers.append(nn.Dropout(dropout_values[0]))

    # Additional hidden layers
    for i in range(1, num_layers):
        layers.append(nn.Linear(neuron_list[i - 1], neuron_list[i]))
        layers.append(nn.ReLU())
        if dropout_values[i] != 0:
            layers.append(nn.Dropout(dropout_values[i]))

    # Always include the final specified layer
    layers.append(nn.Linear(neuron_list[-1] if num_layers > 0 else num_features, num_classes))
    # layers.append(nn.Softmax(dim=1)) not needed cause cross entropy criterion

    return nn.Sequential(*layers)

## Training Setup

In [ ]:
# Setting device for the computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
hyperparams = {
    "num_epochs": 15,
    "batch_size": 64,
    #optimizer
    "opt": "adam",
    "learning_rate": 2e-4,
    #scheduler
    "decay_rate": 0.5,
    #nnet
    "neuron_layer_list": [512, 256],
    "dropout_values": [0.25, 0.25],
}

### Setting up the model using ResNet50 as backbone

In [ ]:
# default `log_dir` is "runs" - we'll be more specific here
model_name = 'ResNet50-vanilla_dataset'
writer = SummaryWriter(f'runs/{model_name}')

test_abs = int(len(train_dataset) * 0.8)
train_subset, val_subset = random_split(
    train_dataset, [test_abs, len(train_dataset) - test_abs])

train_data_size = len(train_subset)

# Create DataLoader instances for training and validation
train_loader = torch.utils.data.DataLoader(
    train_subset,
    batch_size=hyperparams["batch_size"],
    shuffle=True,
    num_workers=0)
val_loader = torch.utils.data.DataLoader(
    val_subset,
    batch_size=hyperparams["batch_size"],
    shuffle=True,
    num_workers=0)
test_loader = DataLoader(
    test_dataset,
    batch_size=hyperparams["batch_size"],
    shuffle=True,
    num_workers=0)

# Model initialization
model = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
for param in model.parameters():
    param.requires_grad = False
# Define the layers you want to add
model.fc = create_dynamic_network(model.fc.in_features, num_classes=43, neuron_list=hyperparams["neuron_layer_list"],
                                  dropout_values=hyperparams["dropout_values"])
writer.add_graph(model, torch.rand(1, 3, 224, 224))
model = model.to(device)

# Define loss function, optimizer, etc.
criterion = nn.CrossEntropyLoss()
# Set optimizers
if hyperparams['opt'] == 'sgd':
    optimizer = torch.optim.SGD(model.fc.parameters(), lr=hyperparams["learning_rate"])
elif hyperparams['opt'] == 'adam':
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=hyperparams["learning_rate"])
elif hyperparams['opt'] == 'rmsprop':
    optimizer = torch.optim.RMSprop(model.fc.parameters(), lr=hyperparams["learning_rate"])
else:
    raise ValueError('Invalid optimizer provided')
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, factor=hyperparams["decay_rate"], min_lr=1e-6,
                                            mode='min', threshold=1e-3)

### Train the model

In [ ]:
# Train model
trained_model = train_model(device=device, model=model, criterion=criterion, optimizer=optimizer,scheduler=scheduler,
                            train_loader=train_loader, val_loader=val_loader, num_epochs=hyperparams["num_epochs"],
                            model_name=model_name)

ta = test_model(trained_model=trained_model, test_loader=test_loader)
writer.add_hparams({key: str(value) if isinstance(value, list) else value for key, value in hyperparams.items()},
                   metric_dict={'Training/Test Accuracy': ta})

# Saving the trained model

In [ ]:
print('Finished Training')
torch.save(trained_model, f'./models/trained_model_{model_name}_final.pth')